In [1]:
import os
import numpy as np

import tensorflow as tf
print("TensorFlow version:", tf.__version__)

np.set_printoptions(precision=4)
seg_length = 500

TensorFlow version: 2.6.0


In [2]:
x = None
y = None

for db in ["chf2db", "chfdb"]:
    for record in os.listdir(f'data/{db}/rr'):
        rr = np.loadtxt(f'data/{db}/rr/{record}')
        n_segments = rr.shape[0] // seg_length
        rr = rr[:n_segments*seg_length].reshape((n_segments, seg_length))
        labels = np.c_[np.ones(n_segments), np.zeros(n_segments)]

        if x is None or y is None:
            x = rr
            y = labels
        else:
            x = np.r_[rr, x]
            y = np.r_[labels, y]


for db in ["nsrdb", "nsr2db", "fantasia"]:
    for record in os.listdir(f'data/{db}/rr'):
        rr = np.loadtxt(f'data/{db}/rr/{record}')
        n_segments = rr.shape[0] // seg_length
        rr = rr[:n_segments*seg_length].reshape((n_segments, seg_length))
        labels = np.c_[np.zeros(n_segments), np.ones(n_segments)]

        if x is None or y is None:
            x = rr
            y = labels
        else:
            x = np.r_[rr, x]
            y = np.r_[labels, y]

x.shape, y.shape

((25670, 500), (25670, 2))

In [3]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit_transform(x)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=410)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=410)

X_train.shape, X_valid.shape, X_test.shape

((20792, 500), (2311, 500), (2567, 500))

In [4]:
np.unique(y_train[:,0], return_counts=True)

(array([0., 1.]), array([12796,  7996], dtype=int64))

In [5]:
inputs = tf.keras.layers.Input(shape=(500, 1))

def inception_lstm(inputs):
    a = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(5, return_sequences=True))(inputs)
    a = tf.keras.layers.LSTM(5, return_sequences=True)(a)

    b = tf.keras.layers.Conv1D(5, kernel_size=1)(inputs)
    b = tf.keras.layers.Conv1D(5, kernel_size=3)(b)

    c = tf.keras.layers.Conv1D(5, kernel_size=1)(inputs)
    c = tf.keras.layers.Conv1D(5, kernel_size=5)(c)

    d = tf.keras.layers.MaxPool1D(3)(inputs)
    d = tf.keras.layers.Conv1D(5, kernel_size=1)(d)

    concat = tf.keras.layers.Concatenate(1)([a, b, c, d])
    return concat

layer1 = inception_lstm(inputs)
layer2 = inception_lstm(layer1)
dropout = tf.keras.layers.Dropout(0.2)(layer2)
flattened = tf.keras.layers.Flatten()(dropout)
output = tf.keras.layers.Dense(2, activation='sigmoid')(flattened)

model = tf.keras.Model(inputs, output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 500, 1)]     0                                            
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 500, 10)      280         input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 500, 5)       10          input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 500, 5)       10          input_1[0][0]                    
______________________________________________________________________________________________

In [6]:
loss_fn = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [7]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
model.fit(x=X_train, y=y_train, validation_data=(X_valid, y_valid),
          batch_size=128, epochs=100, callbacks=[callback])

Epoch 1/100
163/163 [==============================] - 141s 833ms/step - loss: 0.6825 - accuracy: 0.6533 - val_loss: 0.7638 - val_accuracy: 0.6768
Epoch 2/100
119/163 [====================>.........] - ETA: 38s - loss: 0.6099 - accuracy: 0.6817

KeyboardInterrupt: 

In [7]:
model.save('model2.keras')

In [54]:
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test).argmax(axis=1)

In [25]:
print(recall_score(y_test.argmax(axis=1), y_pred))
print(precision_score(y_test.argmax(axis=1), y_pred))
print(accuracy_score(y_test.argmax(axis=1), y_pred))

0.9160206718346253
0.840047393364929
0.8441760810284379


In [14]:
from tensorflow.keras.models import load_model

saved_model = load_model("model4 1.keras", compile=False)

In [17]:
saved_model.summary()

from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score

y_pred = saved_model.predict(X_test).argmax(axis=1)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 500, 1)]     0                                            
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 500, 10)      280         input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 500, 5)       10          input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 500, 5)       10          input_1[0][0]                    
______________________________________________________________________________________________

In [18]:
print(recall_score(y_test.argmax(axis=1), y_pred))
print(precision_score(y_test.argmax(axis=1), y_pred))
print(accuracy_score(y_test.argmax(axis=1), y_pred))

0.9276485788113695
0.6005855290673359
0.584339696143358


Model 2: CNN

In [51]:
from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate
from sklearn.metrics import f1_score, accuracy_score

def get_model():
    nclass = 2
    inp = tf.keras.layers.Input(shape=(500, 1))
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(inp)
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = GlobalMaxPool1D()(img_1)
    img_1 = Dropout(rate=0.2)(img_1)

    dense_1 = Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = Dense(nclass, activation=activations.sigmoid, name="dense_3")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = tf.optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics=['acc'])
    model.summary()
    return model

model = get_model()
file_path = "baseline_cnn.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

model.fit(x=X_train, y=y_train, validation_data=(X_valid, y_valid),
          batch_size=128, epochs=100, callbacks=callbacks_list)

model.load_weights(file_path)


Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 500, 1)]          0         
_________________________________________________________________
conv1d_90 (Conv1D)           (None, 496, 16)           96        
_________________________________________________________________
conv1d_91 (Conv1D)           (None, 492, 16)           1296      
_________________________________________________________________
max_pooling1d_32 (MaxPooling (None, 246, 16)           0         
_________________________________________________________________
dropout_41 (Dropout)         (None, 246, 16)           0         
_________________________________________________________________
conv1d_92 (Conv1D)           (None, 244, 32)           1568      
_________________________________________________________________
conv1d_93 (Conv1D)           (None, 242, 32)           310

ValueError: Classification metrics can't handle a mix of multilabel-indicator and binary targets

In [57]:
pred_test = model.predict(X_test)
pred_test = np.argmax(pred_test, axis=1)

print(recall_score(y_test.argmax(axis=1), pred_test ))
print(precision_score(y_test.argmax(axis=1), pred_test ))
print(accuracy_score(y_test.argmax(axis=1), pred_test ))

0.9450904392764858
0.8718712753277712
0.8831320607713284
